In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [2]:
deliveries = pd.read_csv('deliveries_updated_mens_ipl.csv')
matches = pd.read_csv('matches_updated_mens_ipl.csv')

In [3]:
deliveries.head(1)

,matchId,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed
0,1082591,1,0.1,0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
matches.head(1)

,outcome,event,date,gender,reserve_umpire,team1,toss_winner,umpire2,match_referee,season,team2,winner_runs,balls_per_over,player_of_match,date1,neutralvenue,venue,method,toss_decision,city,tv_umpire,date2,winner,umpire1,match_number,eliminator,winner_wickets,matchId
0,NaN,Indian Premier League,2017-04-05,male,N Pandit,Sunrisers Hyderabad,Royal Challengers Bangalore,NJ Llong,J Srinath,2017,Royal Challengers Bangalore,35.0,6,Yuvraj Singh,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal",NaN,field,Hyderabad,A Deshmukh,NaN,Sunrisers Hyderabad,AY Dandekar,1.0,NaN,NaN,1082591


Player Statics

In [5]:
df = deliveries.copy()
mdf = matches.copy()

In [6]:
df.head(1)

,matchId,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed
0,1082591,1,0.1,0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
mdf.head(1)

,outcome,event,date,gender,reserve_umpire,team1,toss_winner,umpire2,match_referee,season,team2,winner_runs,balls_per_over,player_of_match,date1,neutralvenue,venue,method,toss_decision,city,tv_umpire,date2,winner,umpire1,match_number,eliminator,winner_wickets,matchId
0,NaN,Indian Premier League,2017-04-05,male,N Pandit,Sunrisers Hyderabad,Royal Challengers Bangalore,NJ Llong,J Srinath,2017,Royal Challengers Bangalore,35.0,6,Yuvraj Singh,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal",NaN,field,Hyderabad,A Deshmukh,NaN,Sunrisers Hyderabad,AY Dandekar,1.0,NaN,NaN,1082591


In [8]:
def balls_per_dismissels(balls, dismissels):
    if dismissels > 0:
        return balls/dismissels
    else:
        return balls/1
    
def balls_per_boundaries(balls,boundaries):
    if boundaries > 0:
        return balls/boundaries
    else:
        return balls/1

In [9]:
def playerStatistics(df):
    
    df['isDot'] = df['batsman_runs'].apply(lambda x : 1 if x == 0 else 0)
    df['isOne'] = df['batsman_runs'].apply(lambda x : 1 if x == 1 else 0)
    df['isTwo'] = df['batsman_runs'].apply(lambda x : 1 if x == 2 else 0)
    df['isThree'] = df['batsman_runs'].apply(lambda x : 1 if x == 3 else 0)
    df['isFour'] = df['batsman_runs'].apply(lambda x : 1 if x == 4 else 0)
    df['isSix'] =  df['batsman_runs'].apply(lambda x: 1 if x == 6 else 0)
    
    runs = pd.DataFrame(df.groupby(["batsman"])["batsman_runs"].sum()).reset_index().rename(columns={'batsman_runs':'runs'})
    innings = pd.DataFrame(df.groupby(['batsman'])['matchId'].apply(lambda x : len(list(np.unique(x))))).reset_index().rename(columns = {'matchId' : 'innings'})
    balls = pd. DataFrame(df.groupby(['batsman'])['matchId'].count().reset_index()).rename(columns = {'matchId' : 'balls'})
    dismissels = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count().reset_index()).rename(columns = {'player_dismissed' : 'dismissels'})
    
    dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum().reset_index().rename(columns = {'isDot' : 'dots'}))
    ones = pd.DataFrame(df.groupby(['batsman'])['isOne'].sum().reset_index().rename(columns = {'isOne' : 'ones'}))
    twos = pd.DataFrame(df.groupby(['batsman'])['isTwo'].sum().reset_index().rename(columns = {'isTwo' : 'twos'}))
    threes = pd.DataFrame(df.groupby(['batsman'])['isThree'].sum().reset_index().rename(columns = {'isThree' : 'threes'}))
    fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum().reset_index().rename(columns = {'isFour' : 'fours'}))
    sixes = pd .DataFrame(df.groupby(['batsman'])['isSix'].sum().reset_index().rename(columns = {'isSix' : 'sixes'}))
    
    df = pd.merge(innings,runs, on = 'batsman').merge(balls, on = 'batsman').merge(dismissels, on = 'batsman').merge(dots, on='batsman').merge(ones, on = 'batsman').merge(twos, on = 'batsman').merge(threes, on = 'batsman').merge(fours, on='batsman').merge(sixes, on='batsman')
    
    df['SR'] = df.apply(lambda x : 100*(x['runs']/x['balls']),axis = 1)
    df['RPI'] = df.apply(lambda x : x['runs']/x['innings'], axis = 1)
    df['BPD'] = df.apply(lambda x : balls_per_dismissels(x['balls'], x['dismissels']), axis = 1)
    df['BPB'] = df.apply(lambda x : balls_per_boundaries(x['balls'], (x['fours'] + x['sixes'])), axis = 1)
    
    return df

In [10]:
stat_df = playerStatistics(df)
stat_df.head(2)

,batsman,innings,runs,balls,dismissels,dots,ones,twos,threes,fours,sixes,SR,RPI,BPD,BPB
0,A Ashish Reddy,23,280,196,15,61,83,20,1,16,15,142.857143,12.173913,13.066667,6.322581
1,A Badoni,11,161,139,9,57,53,11,0,11,7,115.827338,14.636364,15.444444,7.722222


In [11]:
comb_df = pd.merge(df, mdf, on = 'matchId', how = 'left')
comb_df.head(2)

,matchId,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed,isDot,isOne,isTwo,isThree,isFour,isSix,outcome,event,date,gender,reserve_umpire,team1,toss_winner,umpire2,match_referee,season,team2,winner_runs,balls_per_over,player_of_match,date1,neutralvenue,venue,method,toss_decision,city,tv_umpire,date2,winner,umpire1,match_number,eliminator,winner_wickets
0,1082591,1,0.1,0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,NaN,Indian Premier League,2017-04-05,male,N Pandit,Sunrisers Hyderabad,Royal Challengers Bangalore,NJ Llong,J Srinath,2017,Royal Challengers Bangalore,35.0,6,Yuvraj Singh,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal",NaN,field,Hyderabad,A Deshmukh,NaN,Sunrisers Hyderabad,AY Dandekar,1.0,NaN,NaN
1,1082591,1,0.2,0,2,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,NaN,Indian Premier League,2017-04-05,male,N Pandit,Sunrisers Hyderabad,Royal Challengers Bangalore,NJ Llong,J Srinath,2017,Royal Challengers Bangalore,35.0,6,Yuvraj Singh,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal",NaN,field,Hyderabad,A Deshmukh,NaN,Sunrisers Hyderabad,AY Dandekar,1.0,NaN,NaN


In [12]:
def MyPhase(over_no):
    if over_no <= 6:
        return "powerplay"
    elif over_no <= 15:
        return "middle"
    else:
        return "death-over"

In [13]:
comb_df['phase'] = comb_df.over.apply(lambda x : MyPhase(x))
comb_df.head(1)

,matchId,inning,over_ball,over,ball,batting_team,bowling_team,batsman,non_striker,bowler,batsman_runs,extras,isWide,isNoBall,Byes,LegByes,Penalty,dismissal_kind,player_dismissed,isDot,isOne,isTwo,isThree,isFour,isSix,outcome,event,date,gender,reserve_umpire,team1,toss_winner,umpire2,match_referee,season,team2,winner_runs,balls_per_over,player_of_match,date1,neutralvenue,venue,method,toss_decision,city,tv_umpire,date2,winner,umpire1,match_number,eliminator,winner_wickets,phase
0,1082591,1,0.1,0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,DA Warner,S Dhawan,TS Mills,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,NaN,Indian Premier League,2017-04-05,male,N Pandit,Sunrisers Hyderabad,Royal Challengers Bangalore,NJ Llong,J Srinath,2017,Royal Challengers Bangalore,35.0,6,Yuvraj Singh,NaN,NaN,"Rajiv Gandhi International Stadium, Uppal",NaN,field,Hyderabad,A Deshmukh,NaN,Sunrisers Hyderabad,AY Dandekar,1.0,NaN,NaN,powerplay


In [14]:
def CustomData(df, current_venue, current_phase, current_opposition):
    df = df[df.venue == current_venue]
    df = df[df.phase == current_phase]
    df = df[df.bowling_team == current_opposition]
    
    df.reset_index(inplace = True)
    
    df['isDot'] = df['batsman_runs'].apply(lambda x : 1 if x == 0 else 0)
    df['isOne'] = df['batsman_runs'].apply(lambda x : 1 if x == 1 else 0)
    df['isTwo'] = df['batsman_runs'].apply(lambda x : 1 if x == 2 else 0)
    df['isThree'] = df['batsman_runs'].apply(lambda x : 1 if x == 3 else 0)
    df['isFour'] = df['batsman_runs'].apply(lambda x : 1 if x == 4 else 0)
    df['isSix'] =  df['batsman_runs'].apply(lambda x: 1 if x == 6 else 0)
    
    runs = pd.DataFrame(df.groupby(["batsman"])["batsman_runs"].sum()).reset_index().rename(columns={'batsman_runs':'runs'})
    innings = pd.DataFrame(df.groupby(['batsman'])['matchId'].apply(lambda x : len(list(np.unique(x))))).reset_index().rename(columns = {'matchId' : 'innings'})
    balls = pd. DataFrame(df.groupby(['batsman'])['matchId'].count().reset_index()).rename(columns = {'matchId' : 'balls'})
    dismissels = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count().reset_index()).rename(columns = {'player_dismissed' : 'dismissels'})
    
    dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum().reset_index().rename(columns = {'isDot' : 'dots'}))
    ones = pd.DataFrame(df.groupby(['batsman'])['isOne'].sum().reset_index().rename(columns = {'isOne' : 'ones'}))
    twos = pd.DataFrame(df.groupby(['batsman'])['isTwo'].sum().reset_index().rename(columns = {'isTwo' : 'twos'}))
    threes = pd.DataFrame(df.groupby(['batsman'])['isThree'].sum().reset_index().rename(columns = {'isThree' : 'threes'}))
    fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum().reset_index().rename(columns = {'isFour' : 'fours'}))
    sixes = pd .DataFrame(df.groupby(['batsman'])['isSix'].sum().reset_index().rename(columns = {'isSix' : 'sixes'}))
    
    df = pd.merge(innings,runs, on = 'batsman').merge(balls, on = 'batsman').merge(dismissels, on = 'batsman').merge(dots, on='batsman').merge(ones, on = 'batsman').merge(twos, on = 'batsman').merge(threes, on = 'batsman').merge(fours, on='batsman').merge(sixes, on='batsman')
    
    df['SR'] = df.apply(lambda x : 100*(x['runs']/x['balls']),axis = 1)
    df['RPI'] = df.apply(lambda x : x['runs']/x['innings'], axis = 1)
    df['BPD'] = df.apply(lambda x : balls_per_dismissels(x['balls'], x['dismissels']), axis = 1)
    df['BPB'] = df.apply(lambda x : balls_per_boundaries(x['balls'], (x['fours'] + x['sixes'])), axis = 1)
    df['DPB'] = df.apply(lambda x : (x['dots']/x['balls']), axis = 1)
    
    return df

In [15]:
new_df = CustomData(comb_df, "MA Chidambaram Stadium, Chepauk", "middle", "Chennai Super Kings")
new_df.head(2)

,batsman,innings,runs,balls,dismissels,dots,ones,twos,threes,fours,sixes,SR,RPI,BPD,BPB,DPB
0,A Ashish Reddy,1,1,2,0,1,1,0,0,0,0,50.000000,1.0,2.0,2.0,0.500000
1,A Mishra,1,6,11,1,6,4,1,0,0,0,54.545455,6.0,11.0,11.0,0.545455


In [16]:
ing_df1=new_df[(new_df.runs>=20)&(new_df.innings>1)]
ing_df1.head()

,batsman,innings,runs,balls,dismissels,dots,ones,twos,threes,fours,sixes,SR,RPI,BPD,BPB,DPB
5,AB de Villiers,3,99,67,0,17,29,7,2,11,1,147.761194,33.0,67.0,5.583333,0.253731
7,AD Mathews,2,52,43,0,13,22,3,0,3,2,120.930233,26.0,43.0,8.600000,0.302326
10,AL Menaria,2,23,32,1,15,13,3,0,1,0,71.875000,11.5,32.0,32.000000,0.468750
14,AT Rayudu,4,28,40,2,13,26,1,0,0,0,70.000000,7.0,20.0,40.000000,0.325000
25,CL White,2,41,41,1,16,19,1,0,5,0,100.000000,20.5,41.0,8.200000,0.390244


In [17]:
wt_sr,wt_rpi,wt_bpd,wt_dot_percentage=0.13,0.27,0.16,0.45

In [18]:
ing_df1['calc_SR'] = ing_df1['SR'].apply(lambda x: x*x) 
ing_df1['calc_RPI'] = ing_df1['RPI'].apply(lambda x: x*x) 
ing_df1['calc_BPD'] = ing_df1['BPD'].apply(lambda x: x*x) 
ing_df1['calc_dot_percentage'] = ing_df1['DPB'].apply(lambda x: x*x)

sq_sr, sq_rpi, sq_bpd, sq_dot_percentage = np.sqrt(ing_df1[['calc_SR','calc_RPI', 'calc_BPD', 'calc_dot_percentage']].sum(axis = 0))

ing_df1['calc_SR'] = ing_df1['calc_SR'].apply(lambda x: x/sq_sr) 
ing_df1['calc_RPI'] = ing_df1['calc_RPI'].apply(lambda x: x/sq_rpi) 
ing_df1['calc_BPD'] = ing_df1['calc_BPD'].apply(lambda x: x/sq_bpd) 
ing_df1['calc_dot_percentage'] = ing_df1['calc_dot_percentage'].apply(lambda x: x/sq_dot_percentage)

ing_df1['calc_SR'] = ing_df1['calc_SR'].apply(lambda x: x*wt_sr) 
ing_df1['calc_RPI'] = ing_df1['calc_RPI'].apply(lambda x: x*wt_rpi) 
ing_df1['calc_BPD'] = ing_df1['calc_BPD'].apply(lambda x: x*wt_bpd) 
ing_df1['calc_dot_percentage'] = ing_df1['calc_dot_percentage'].apply(lambda x: x*wt_dot_percentage)

best_sr, worst_sr = max(ing_df1['calc_SR']), min(ing_df1['calc_SR'])
best_rpi, worst_rpi = max(ing_df1['calc_RPI']), min(ing_df1['calc_RPI'])
best_bpd, worst_bpd = max(ing_df1['calc_BPD']), min(ing_df1['calc_BPD'])
best_dot_percentage, worst_dot_percentage = min(ing_df1['calc_dot_percentage']), max(ing_df1['calc_dot_percentage'])

In [19]:
ing_df1['dev_best_SR'] = ing_df1['calc_SR'].apply(lambda x: (x-best_sr)*(x-best_sr)) 
ing_df1['dev_best_RPI'] = ing_df1['calc_RPI'].apply(lambda x: (x-best_rpi)*(x-best_rpi)) 
ing_df1['dev_best_BPD'] = ing_df1['calc_BPD'].apply(lambda x: (x-best_bpd)*(x-best_bpd)) 
ing_df1['dev_best_dot_percentage'] = ing_df1['calc_dot_percentage'].apply(lambda x: (x-best_dot_percentage)*(x-best_dot_percentage))

ing_df1['dev_best_sqrt'] = ing_df1.apply(lambda x: x['dev_best_SR'] + x['dev_best_RPI'] + x['dev_best_BPD'] + x['dev_best_dot_percentage'], axis = 1) 

ing_df1['dev_worst_SR'] = ing_df1['calc_SR'].apply(lambda x: (x-worst_sr)*(x-worst_sr)) 
ing_df1['dev_worst_RPI'] = ing_df1['calc_RPI'].apply(lambda x: (x-worst_rpi)*(x-worst_rpi)) 
ing_df1['dev_worst_BPD'] = ing_df1['calc_BPD'].apply(lambda x: (x-worst_bpd)*(x-worst_bpd)) 
ing_df1['dev_worst_dot_percentage'] = ing_df1['calc_dot_percentage'].apply(lambda x: (x-worst_dot_percentage)*(x-worst_dot_percentage))

ing_df1['dev_worst_sqrt'] = ing_df1.apply(lambda x: x['dev_worst_SR'] + x['dev_worst_RPI'] + x['dev_worst_BPD'] + x['dev_worst_dot_percentage'], axis = 1)

In [20]:
ing_df1['score'] = ing_df1.apply(lambda x: x['dev_worst_sqrt']/(x['dev_worst_sqrt'] + x['dev_best_sqrt']), axis = 1)

In [28]:
ing_df1[['batsman', 'score', 'SR']].sort_values(by = 'score', ascending = False).head().reset_index(drop = True)

,batsman,score,SR
0,SE Marsh,0.885897,186.000000
1,AB de Villiers,0.835405,147.761194
2,G Gambhir,0.560716,111.111111
3,NV Ojha,0.469076,163.414634
4,SR Watson,0.468230,159.090909


From the above score its pretty know that Shaun Marsh, AB de Villers, Gautam Gambhir, Naman Ojha, Shane Watson are the top 5 batsman who played the Chennai Super kings spinner's well in MA Chidambaram Stadium. 